<a href="https://colab.research.google.com/github/juniperlovato/AASP/blob/main/2024_Network_HigheredAIPolicies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:

import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [86]:
#DFV2 scraped text
df = pd.read_csv('/content/drive/MyDrive/RESEARCH/PROJECTS/2024_GenAI_Higher_Ed/2024_ScapeClean_Text_HigheredAIPolicies.csv')
dfcopy = df.copy()

In [87]:
# Text_1_URL_Resource, Text_1_URL_Guideline,Text_1_URL_Policy2,Text_1_URL_Policy1s,Text_3_Admissions
dfcopy.columns

Index(['Random_Number', 'display_name', 'R1_cc', 'US_State', 'Type_x',
       'Category', 'wiki_url', 'Students_Wiki_DataUSA',
       'Endowment_millions_Wiki_DataUSA', 'Established_Wiki',
       'Have course catalogs', 'ror_id', '0_URL_AI_Task_force',
       '1_URL_Policy1', '1_URL_Policy2', '1_Type_of_policy_1',
       '1_Type_of_policy_2', '1_URL_Guideline', '1_URL_Resource',
       '2_Student_Cheating_Policy', '3_Admissions', 'Type_Wiki',
       'Location_Wiki', 'Parent_institution_Wiki', 'President_Wiki',
       'Campus_Wiki', 'Colors_Wiki', 'Nickname_Wiki', 'Website_Wiki',
       'Former_names_Wiki', 'Motto_Wiki', 'Academic_affiliations_Wiki',
       'Budget_Wiki', 'Sporting_affiliations_Wiki', 'Mascot_Wiki',
       'Former_name_Wiki', 'Motto_in_English_Wiki', 'Provost_Wiki',
       'Academic_staff_Wiki', 'Undergraduates_Wiki', 'Postgraduates_Wiki',
       'Chancellor_Wiki', 'Founder_Wiki', 'Accreditation_Wiki',
       'Newspaper_Wiki', 'Affiliations_Wiki', 'Academic_affiliation_

In [88]:
dfcopy['Text_1_URL_Resource']

,Text_1_URL_Resource
0,[]
1,[]
2,"[""resources for instructors - generative ai: c..."
3,[]
4,[]
...,...
834,[]
835,[]
836,[]
837,[]


In [99]:
#make column display_name lowercase
dfcopy['display_name'] = dfcopy['display_name'].str.lower()
dfcopy['display_name']

,display_name
0,longwood university
1,abraham baldwin agricultural college
2,pennsylvania state university
3,university of arkansas for medical sciences
4,appalachian state university
...,...
834,worcester state university
835,yale university
836,yeshiva university
837,york college


In [105]:
# if the text in any of these columns (Text_1_URL_Resource, Text_1_URL_Guideline,Text_1_URL_Policy2,Text_1_URL_Policy1s,Text_3_Admissions) contains the string of any item in column named 'display_name' other than its own then return a list of the string in a new column called edges

def find_edges(df, display_name_col, text_columns):
    """
    Finds instances where text in specified columns contains strings
    from the 'display_name' column, excluding self-matches.

    Args:
        df: The input DataFrame.
        display_name_col: The name of the column containing display names.
        text_columns: A list of column names to search for display name strings.

    Returns:
        A list of strings representing the edges found.
    """
    all_edges = []  # Initialize a list to store all edges for each row

    for index, row in df.iterrows():
        edges = []  # Initialize an empty list for each row's edges
        display_name = row[display_name_col]
        for col in text_columns:
            if pd.notna(row[col]):  # Check for NaN values
                text = str(row[col])  # Convert to string to avoid errors
                for other_index, other_row in df.iterrows():
                    other_display_name = other_row[display_name_col]
                    if other_display_name != display_name and other_display_name in text:
                        edges.append(other_display_name)
        all_edges.append(edges)  # Append the list of edges for this row to all_edges

    return all_edges  # Return the list of lists (edges for all rows)

display_name_column = 'display_name'
text_cols = ['Text_1_URL_Resource', 'Text_1_URL_Guideline', 'Text_1_URL_Policy2', 'Text_1_URL_Policy1', 'Text_3_Admissions']
dfcopy['edges'] = find_edges(dfcopy, display_name_column, text_cols)
dfcopy['edges']

,edges
0,[]
1,[]
2,"[the pennsylvania state university, the pennsy..."
3,[university of arkansas]
4,[]
...,...
834,[]
835,[]
836,[]
837,[]


In [106]:
dfcopy[['edges','display_name']]

,edges,display_name
0,[],longwood university
1,[],abraham baldwin agricultural college
2,"[the pennsylvania state university, the pennsy...",pennsylvania state university
3,[university of arkansas],university of arkansas for medical sciences
4,[],appalachian state university
...,...,...
834,[],worcester state university
835,[],yale university
836,[],yeshiva university
837,[],york college


In [103]:
print(dfcopy.iloc[0]['edges'])

[]


In [107]:
#create a nodes and edges list for a network analysis in networkx using  dfcopy['display_name'] as the nodes and dfcopy['edges'] as the edges and then return to a new dataframe

# Create an empty list to store edges
edges_list = []

# Iterate through the DataFrame rows
for index, row in dfcopy.iterrows():
  display_name = row['display_name']
  if isinstance(row['edges'], list):
    for edge in row['edges']:
        # Add the edge to the list
        edges_list.append((display_name, edge))


# Create a new DataFrame with nodes and edges
network_df = pd.DataFrame({'source': [edge[0] for edge in edges_list],
                           'target': [edge[1] for edge in edges_list]})

network_df

,source,target
0,pennsylvania state university,the pennsylvania state university
1,pennsylvania state university,the pennsylvania state university
2,university of arkansas for medical sciences,university of arkansas
3,west virginia university,university of washington
4,auburn university,auburn university at montgomery
...,...,...
546,cuny graduate school and university center,john jay college of criminal justice
547,cuny graduate school and university center,lehman college
548,cuny graduate school and university center,new york city college of technology
549,cuny graduate school and university center,queens college


In [108]:
#exporting the nodes and edges and then making the final visualization in Gephi
network_df.to_csv('/content/drive/MyDrive/RESEARCH/PROJECTS/2024_GenAI_Higher_Ed/2024_Network_HigheredAIPolicies.csv', index=False)

#in Gephi
# citation network structure where node size is proportional to in-degree and color set-up to page rank centrality (you want to give yourself a higher centrality based on the centrality of the people that reference you, and with a baseline centrality for institution with in-degree = 0)
# % In this setup:
# % Universities with high in-degree (larger nodes) but low PageRank (less central color) are receiving many citations, but possibly from less influential institutions.
# % Universities with low in-degree but high PageRank could be cited by highly influential universities, making them important within the network despite having fewer direct citations.
# % For those universities with in-degree = 0, they are given a baseline PageRank centrality, meaning they are considered in the network, but their centrality doesn’t grow as they are not cited by any others.

